# 人物検出のサンプルコード

In [3]:
# ライブラリのインポート
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import glob

## 対象画像とマスク画像のフォルダ（**要修正**）

In [4]:
# 対象画像のフォルダ（適宜修正すること）
TARGET_FOLDER = "./images/"

# マスク画像を保存するフォルダ（適宜修正すること）
MASK_FOLDER = "./results/" 

## マスク画像を生成する関数（**要修正**）
### 【改良したアルゴリズムの説明をここに記載すること】

In [5]:
# 画像から人物のマスクを生成する関数（適宜修正すること）
def detect(image):
    
    # グレースケールに変換
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 閾値127で2値化
    _, image_binary = cv2.threshold(image_gray, 127, 255, cv2.THRESH_BINARY)

    # モルフォロジー処理
    kernel = np.ones((5,5),np.uint8)
    image_mask = cv2.morphologyEx(image_binary, cv2.MORPH_OPEN, kernel, iterations=3)
    image_mask = cv2.morphologyEx(image_binary, cv2.MORPH_CLOSE, kernel, iterations=3)
    
    return image_mask

In [6]:
# 画像から人物のマスクを生成する関数（HOG特徴量）
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

def detect_hog(image):
    
    # RGBに変換
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 人物検出
    (rects, weights) = hog.detectMultiScale(image_rgb, winStride=(8, 8), padding=(8, 8), scale=1.05)

    # 画像と同じサイズのnumpy配列を作成
    height, width, channels = image_rgb.shape
    image_mask = np.zeros((height, width), dtype=np.uint8)

   # 5. 検出された矩形を描画
    for (x, y, w, h) in rects:
        cv2.rectangle(image_mask, (x, y), (x + w, y + h), 255, -1)
    
    return image_mask

In [ ]:
# 画像から人物のマスクを生成する関数（YOLO）

# YOLOの設定ファイルと重みファイルを読み込む
config_path = "yolo/yolov4-p6.cfg"
weights_path = "yolo/yolov4-p6.weights"
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)

# ラベル（クラス）の読み込み
labels = open("yolo/coco.names").read().strip().split("\n")

# 出力レイヤーを取得
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

def detect_yolo(image):
    # RGBに変換
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 画像サイズ
    height, width = image.shape[:2]

    # 画像の前処理
    blob = cv2.dnn.blobFromImage(image_rgb, scalefactor=1/255.0, size=(640, 640), swapRB=True, crop=False)
    net.setInput(blob)

    # 人物検出
    detections = net.forward(output_layers)

    # 画像と同じサイズのグレースケールマスクを作成
    image_mask = np.zeros((height, width), dtype=np.uint8)

    # 検出結果を解析
    for output in detections:
        for detection in output:
            scores = detection[5:]  # 各クラスのスコアを取得
            class_id = np.argmax(scores)  # 最も高いスコアを持つクラスID
            confidence = scores[class_id]  # そのクラスの信頼度

            if class_id == 0 and confidence > 0.5:  # 人物クラスかつ信頼度が0.5以上
                box = detection[0:4] * np.array([width, height, width, height])
                (center_x, center_y, box_width, box_height) = box.astype("int")

                # バウンディングボックスの左上座標を計算
                x = int(center_x - box_width / 2)
                y = int(center_y - box_height / 2)

                # マスクに矩形を描画
                cv2.rectangle(image_mask, (x, y), (x + box_width, y + box_height), 255, -1)

    return image_mask

## 対象画像のマスク画像を生成（修正不可）

In [9]:
# マスク画像のフォルダが存在する場合は削除
if os.path.exists(MASK_FOLDER):
    shutil.rmtree(MASK_FOLDER)

# マスク画像の保存フォルダの作成
os.makedirs(MASK_FOLDER)

# 画像一覧の取得
files = sorted(glob.glob(TARGET_FOLDER + "*.png"))
for file in files:
    image = cv2.imread(file) # 画像の読み込み
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # BGRからRGBに変換

    #image_mask = detect(image_rgb) # マスク画像を生成
    #image_mask = detect_hog(image_rgb) # マスク画像を生成
    image_mask = detect_yolo(image_rgb) # マスク画像を生成

    filename = os.path.basename(file)
    cv2.imwrite(f"{MASK_FOLDER}{filename}", image_mask) # マスク画像を保存